In [ ]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D
from keras.layers import BatchNormalization, Concatenate
from keras.optimizers import SGD

import numpy as nps
#from sklearn.model_selection import train_test_split
import skimage.io as sio
import skimage.color as scolor
from skimage.transform import rescale, resize, downscale_local_mean
from matplotlib import pyplot as plt

#import pickle
#import os
#import json
#import glob
import sys
import h5py

#import keras_data_reader as dr

In [ ]:
hdf_filename = "datasets/processed/aug_structured_data-liver_only.hdf5"
hdf_file = h5py.File(hdf_filename, 'r')
train_data = hdf_file['train_data']
train_labels = hdf_file["train_labels"]
val_data = hdf_file['val_data']
val_labels = hdf_file["val_labels"]

In [ ]:
inputs = Input(shape=(240, 232, 1))

# downsampling
xc1 = Conv2D(32, (3, 3), padding='same', activation='relu', strides=(1, 1))(inputs)
xc1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(xc1)
xc2 = Conv2D(64, (3, 3), padding='same', activation='relu', strides=(1, 1))(xc1)
xc2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(xc2)
xmp3 = MaxPooling2D(pool_size=(2, 2))(xc2)

xc1b = Conv2D(128, (3, 3), padding='same', activation='relu', strides=(1, 1))(xmp3)
xc1b = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(xc1b)
xc2b = Conv2D(256, (3, 3), padding='same', activation='relu', strides=(1, 1))(xc1b)
xc2b = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(xc2b)
xmp3b = MaxPooling2D(pool_size=(2, 2))(xc2b)

# upsampling
xup4 = UpSampling2D(size=(2, 2), data_format=None)(xmp3b)

concat1 = Concatenate(axis=-1)([xup4, xc2b])

xct5 = Conv2DTranspose(256, (3, 3), strides=(1, 1), padding='same', data_format=None, activation='relu')(xup4)
xct5 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(concat1)
xct6 = Conv2DTranspose(128, (3, 3), strides=(1, 1), padding='same', data_format=None, activation='relu')(xct5)
xct6 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(xct6)

# upsampling
xup4b = UpSampling2D(size=(2, 2), data_format=None)(xct6)

concat2 = Concatenate(axis=-1)([xup4b, xc2])

xct5b = Conv2DTranspose(64, (3, 3), strides=(1, 1), padding='same', data_format=None, activation='relu')(concat2)
xct5b = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(xct5b)
xct6b = Conv2DTranspose(32, (3, 3), strides=(1, 1), padding='same', data_format=None, activation='relu')(xct5b)
xct6b = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(xct6b)

predictions = Conv2D(3, (1, 1), padding='same', activation='softmax')(xct6b)
#predictions = BatchNormalization()(predictions)

In [ ]:
model = Model(inputs=inputs, outputs=predictions)

sgd = SGD(lr=0.01)#, clipvalue=0.5)
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())

In [ ]:
#sys.exit(0)
epochs = 5
#labels = np.array([1,1,1])
model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=epochs, batch_size=8, shuffle='batch')

In [ ]:
model_filename = "models/segnet-20epoch_aug_structured_data-liver_only.hdf5"
model.save(model_filename)

# Ulozeni vysledku

In [ ]:
test_data = hdf_file['test_data']
test_labels = hdf_file["test_labels"]

In [ ]:
test_predicted_labels = model.predict(test_data, batch_size=8)

In [ ]:
test_results_path = "test_results-20epoch_aug_structured_data-liver_only.hdf5"
hdf5_file = h5py.File(test_results_path , mode='w')
hdf5_file.create_dataset("test_data", test_data.shape, np.int8)
hdf5_file["test_data"][...] = test_data
hdf5_file.create_dataset("test_labels", test_labels.shape, np.int8)
hdf5_file["test_labels"][...] = test_labels
hdf5_file.create_dataset("test_predictions", test_predicted_labels.shape, np.float)
hdf5_file["test_predictions"][...] = test_predicted_labels
hdf5_file.close()

In [ ]:
hdf5_file.close()

In [ ]:
res = test_predicted_labels[39]
plt.imshow(res)
plt.colorbar()
plt.show()

In [ ]:
res = test_predicted_labels[39]
print(res.shape)
plt.imshow(res[:,:,1], cmap="gray")
plt.colorbar()
plt.show()

In [ ]:
res = test_predicted_labels[39]
print(res.shape)
plt.imshow(res[:,:,1]>0.33, cmap="gray")
plt.colorbar()
plt.show()

In [ ]:
lab = test_labels[39].astype(float)
plt.imshow(lab)
plt.colorbar()
plt.show()

In [ ]:
res = test_data[39]
print(res.shape)
plt.imshow(res[:,:,0], cmap="gray")
plt.colorbar()
plt.show()